In [ ]:
# Implement sequence modeling of stock prices using LSTM layer in Bitcoin minute price data
# Comparing accuracy of different timesteps of 60, 120 and 180 

In [1]:
import tensorflow as tf

tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [4]:
#changing the working directory

%cd /content/gdrive/My Drive/Kaggle


/content/gdrive/My Drive/Kaggle


In [5]:
% pwd

#Check the present working directory using pwd command

'/content/gdrive/My Drive/Kaggle'

In [6]:
!ls

 0.2.0.zip				  Recommendation_df_test1.csv
 Apple-Twitter-Sentiment-DFE.csv	  Recommendation_df_test.csv
 apple-twitter-sentiment-texts.csv	  Recommendation_df_train1.csv
 Bitcoin_prediction_One_hot_encoded.csv   Recommendation_df_train.csv
 BitCoinPrice.csv			  sampleSubmission.csv
 Bitcoin_tweet_sentiment.csv		  sampleSubmission.csv.zip
 BTCUSD_1hr.csv				  sampleSubmission_v2.csv.zip
 BTCUSD_1min.csv			  saved_models
 BTCUSD_day.csv				  Song_Recommendation_Model.h5
 Credit_Card_Autoencoder_Model.h5	  submission.csv
 creditcard.csv				  test.csv
 decoder_layer.png			  test_embed_df.csv
 decoder.png				  test.tsv
 encoder_layer.png			  train.csv
 encoder.png				  train_embed_df.csv
 FastText				  train-sample.csv.zip
'Harry Potter 1.csv'			  train.tsv
'Harry Potter 2.csv'			  train_v2.csv
'Harry Potter 3.csv'			  transformer.png
 kaggle.json				  tweetsfinal1.csv
 Movie_Conversation.csv			  tweetsfinal2.csv
 movie_conversations.txt		  tweetsfinal3.csv
 Movie_Dialogue_Model1.h5		  tw

In [105]:
df_bc1 = pd.read_csv('BTCUSD_1min.csv')
df_bc1

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1586390400000,2020-04-09 00:00:00,BTCUSD,7369.60,7369.60,7348.47,7350.41,7.733528
1,1586390340000,2020-04-08 23:59:00,BTCUSD,7364.37,7369.60,7364.37,7369.60,1.310954
2,1586390280000,2020-04-08 23:58:00,BTCUSD,7360.89,7367.18,7360.89,7364.37,1.018774
3,1586390220000,2020-04-08 23:57:00,BTCUSD,7357.62,7366.12,7357.62,7360.89,0.026251
4,1586390160000,2020-04-08 23:56:00,BTCUSD,7370.71,7370.71,7357.62,7357.62,0.444867
...,...,...,...,...,...,...,...,...
2283514,1444311840,2015-10-08 13:44:00,BTCUSD,242.96,242.96,242.96,242.96,0.033491
2283515,1444311780,2015-10-08 13:43:00,BTCUSD,242.95,242.96,242.95,242.96,0.010000
2283516,1444311720,2015-10-08 13:42:00,BTCUSD,242.95,242.95,242.95,242.95,0.000000
2283517,1444311660,2015-10-08 13:41:00,BTCUSD,242.50,242.95,242.50,242.95,0.001000


In [106]:
# Finding out percentage increase or decrease 

df_bc1['Pct_change'] = df_bc1['Close'].pct_change()
df_bc1

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Pct_change
0,1586390400000,2020-04-09 00:00:00,BTCUSD,7369.60,7369.60,7348.47,7350.41,7.733528,NaN
1,1586390340000,2020-04-08 23:59:00,BTCUSD,7364.37,7369.60,7364.37,7369.60,1.310954,0.002611
2,1586390280000,2020-04-08 23:58:00,BTCUSD,7360.89,7367.18,7360.89,7364.37,1.018774,-0.000710
3,1586390220000,2020-04-08 23:57:00,BTCUSD,7357.62,7366.12,7357.62,7360.89,0.026251,-0.000473
4,1586390160000,2020-04-08 23:56:00,BTCUSD,7370.71,7370.71,7357.62,7357.62,0.444867,-0.000444
...,...,...,...,...,...,...,...,...,...
2283514,1444311840,2015-10-08 13:44:00,BTCUSD,242.96,242.96,242.96,242.96,0.033491,0.000000
2283515,1444311780,2015-10-08 13:43:00,BTCUSD,242.95,242.96,242.95,242.96,0.010000,0.000000
2283516,1444311720,2015-10-08 13:42:00,BTCUSD,242.95,242.95,242.95,242.95,0.000000,-0.000041
2283517,1444311660,2015-10-08 13:41:00,BTCUSD,242.50,242.95,242.50,242.95,0.001000,0.000000


In [107]:
# Putting 1 if percentage increase and 0 if percentage decrease/stays same
# Idea being invest in stock if 1 and dont invest / short the stock if 0

df_bc1['Increase'] = df_bc1['Pct_change'].apply(lambda x: 1 if x > 0 else 0)
df_bc1

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Pct_change,Increase
0,1586390400000,2020-04-09 00:00:00,BTCUSD,7369.60,7369.60,7348.47,7350.41,7.733528,NaN,0
1,1586390340000,2020-04-08 23:59:00,BTCUSD,7364.37,7369.60,7364.37,7369.60,1.310954,0.002611,1
2,1586390280000,2020-04-08 23:58:00,BTCUSD,7360.89,7367.18,7360.89,7364.37,1.018774,-0.000710,0
3,1586390220000,2020-04-08 23:57:00,BTCUSD,7357.62,7366.12,7357.62,7360.89,0.026251,-0.000473,0
4,1586390160000,2020-04-08 23:56:00,BTCUSD,7370.71,7370.71,7357.62,7357.62,0.444867,-0.000444,0
...,...,...,...,...,...,...,...,...,...,...
2283514,1444311840,2015-10-08 13:44:00,BTCUSD,242.96,242.96,242.96,242.96,0.033491,0.000000,0
2283515,1444311780,2015-10-08 13:43:00,BTCUSD,242.95,242.96,242.95,242.96,0.010000,0.000000,0
2283516,1444311720,2015-10-08 13:42:00,BTCUSD,242.95,242.95,242.95,242.95,0.000000,-0.000041,0
2283517,1444311660,2015-10-08 13:41:00,BTCUSD,242.50,242.95,242.50,242.95,0.001000,0.000000,0


In [142]:
# Timesteps or window size = 60

total = df_bc1['Increase'].tolist()
total = np.array(total)
total1 = total[:2283480]
len(total1)


2283480

In [143]:
total1 = total1.reshape(-1,60,1)
total1.shape

(38058, 60, 1)

In [144]:
split = int(0.8*len(total1))
split

30446

In [145]:
xx = total1[:split]
xx_test = total1[split:]

In [146]:
xx.shape

(30446, 60, 1)

In [147]:
xx_test.shape

(7612, 60, 1)

In [150]:
[x, y] = np.hsplit(xx, np.array([59]))

In [151]:
x.shape

(30446, 59, 1)

In [152]:
y.shape

(30446, 1, 1)

In [153]:
[x_test, y_test] = np.hsplit(xx_test, np.array([59]))

In [156]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

n_steps = 59
n_features = 1

model1 = Sequential()
model1.add(LSTM(50, return_sequences=True, input_shape=(59,1), activation='relu'))
model1.add(LSTM(50, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 59, 50)            10400     
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [158]:
model1.fit(x, y, batch_size = 32, epochs=5, verbose = 1)

Epoch 1/5
952/952 [==============================] - 321s 337ms/step - loss: 0.5617 - accuracy: 0.7138
Epoch 2/5
952/952 [==============================] - 326s 343ms/step - loss: 0.5526 - accuracy: 0.7138
Epoch 3/5
952/952 [==============================] - 327s 343ms/step - loss: 0.5511 - accuracy: 0.7139
Epoch 4/5
952/952 [==============================] - 323s 339ms/step - loss: 0.5506 - accuracy: 0.7137
Epoch 5/5
952/952 [==============================] - 324s 340ms/step - loss: 0.5499 - accuracy: 0.7138


In [159]:
model1.evaluate(x_test, y_test, verbose = 1)

238/238 [==============================] - 7s 28ms/step - loss: 0.1547 - accuracy: 0.9603


[0.1547277867794037, 0.9603257775306702]

In [160]:
model1.metrics_names

['loss', 'accuracy']

In [161]:
# Timesteps or window size = 120

total = df_bc1['Increase'].tolist()
total = np.array(total)
total1 = total[:2283480]
len(total1)


2283480

In [162]:
total1 = total1.reshape(-1,120,1)
total1.shape

(19029, 120, 1)

In [163]:
split = int(0.8*len(total1))
split

15223

In [164]:
xx = total1[:split]
xx_test = total1[split:]

In [165]:
xx.shape

(15223, 120, 1)

In [166]:
xx_test.shape

(3806, 120, 1)

In [167]:
[x, y] = np.hsplit(xx, np.array([119]))

In [168]:
x.shape

(15223, 119, 1)

In [169]:
y.shape

(15223, 1, 1)

In [170]:
[x_test, y_test] = np.hsplit(xx_test, np.array([119]))

In [171]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

n_steps = 119
n_features = 1

model1 = Sequential()
model1.add(LSTM(50, return_sequences=True, input_shape=(119,1), activation='relu'))
model1.add(LSTM(50, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 119, 50)           10400     
                                                                 
 lstm_7 (LSTM)               (None, 50)                20200     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [172]:
model1.fit(x, y, batch_size = 32, epochs=5, verbose = 1)

Epoch 1/5
476/476 [==============================] - 315s 656ms/step - loss: 0.6607 - accuracy: 0.7127
Epoch 2/5
476/476 [==============================] - 306s 642ms/step - loss: 14705801216.0000 - accuracy: 0.7131
Epoch 3/5
476/476 [==============================] - 312s 655ms/step - loss: 374877.1562 - accuracy: 0.7131
Epoch 4/5
476/476 [==============================] - 312s 656ms/step - loss: 0.6037 - accuracy: 0.7131
Epoch 5/5
476/476 [==============================] - 313s 658ms/step - loss: 0.5976 - accuracy: 0.7131


In [173]:
model1.evaluate(x_test, y_test, verbose = 1)

119/119 [==============================] - 6s 50ms/step - loss: 0.3709 - accuracy: 0.9601


[0.37092843651771545, 0.9600630402565002]

In [ ]:
model1.metrics_names

['loss', 'accuracy']

In [174]:
# Timesteps or window size = 180

total = df_bc1['Increase'].tolist()
total = np.array(total)
total1 = total[:2283480]
len(total1)


2283480

In [175]:
total1 = total1.reshape(-1,180,1)
total1.shape

(12686, 180, 1)

In [176]:
split = int(0.8*len(total1))
split

10148

In [177]:
xx = total1[:split]
xx_test = total1[split:]

In [178]:
xx.shape

(10148, 180, 1)

In [179]:
xx_test.shape

(2538, 180, 1)

In [180]:
[x, y] = np.hsplit(xx, np.array([179]))

In [181]:
x.shape

(10148, 179, 1)

In [182]:
y.shape

(10148, 1, 1)

In [183]:
[x_test, y_test] = np.hsplit(xx_test, np.array([179]))

In [184]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

n_steps = 179
n_features = 1

model1 = Sequential()
model1.add(LSTM(50, return_sequences=True, input_shape=(179,1), activation='relu'))
model1.add(LSTM(50, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 179, 50)           10400     
                                                                 
 lstm_9 (LSTM)               (None, 50)                20200     
                                                                 
 dense_4 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [185]:
model1.fit(x, y, batch_size = 32, epochs=5, verbose = 1)

Epoch 1/5
318/318 [==============================] - 311s 970ms/step - loss: 2021261.1250 - accuracy: 0.7118
Epoch 2/5
318/318 [==============================] - 310s 974ms/step - loss: 2386900992.0000 - accuracy: 0.7119
Epoch 3/5
318/318 [==============================] - 310s 973ms/step - loss: 0.7620 - accuracy: 0.7119
Epoch 4/5
318/318 [==============================] - 310s 974ms/step - loss: 0.5953 - accuracy: 0.7119
Epoch 5/5
318/318 [==============================] - 309s 971ms/step - loss: 0.5945 - accuracy: 0.7119


In [186]:
model1.evaluate(x_test, y_test, verbose = 1)

80/80 [==============================] - 6s 74ms/step - loss: 0.3683 - accuracy: 0.9543


[0.3682502806186676, 0.954294741153717]

In [187]:
model1.metrics_names

['loss', 'accuracy']